In [2]:
import pandas as pd


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
data = pd.read_csv('통합데이터.csv', encoding='UTF-8')
data.head()

,question,selection,passage,answer,explanation,q_type,subject_name,q_sub
0,1. 윗글을 이해한 내용으로 적절하지 않은 것은?,①독서 전략을 선택할 때 독서의 목표를 고려할 필요가 있다.\n②독서 전략의 선택을...,독서는 독자가 목표한 결과에 도달하기 위해 글을 읽고 의미를 구성하는 인지 행위이...,⑤,"정답해설 : 3문단에 따르면, 초인지는 문제를 해결하기 위해 독서 전략을 점검하고\...",세부 내용 파악,국어,1_1
1,2. [A]에서 알 수 있는 내용으로 가장 적절한 것은?,①독서 진행 중 이해한 내용을 정리하는 것은 독자 스스로 독서 진행의 문제를 점검하...,독서는 독자가 목표한 결과에 도달하기 위해 글을 읽고 의미를 구성하는 인지 행위이...,③,정답해설 : [A]에서 독서가 중단 없이 이어지는 상태이지만 문제가 발생한 것을 독...,세부 내용 추론,국어,1_2
2,4. ㉠에 대한 설명으로 가장 적절한 것은?,①선거 기간의 후반기에 비해 전반기에 더 많다.\n②시청자와 방송사의 상반된 이해관...,㉠[경마식 보도]는 경마 중계를 하듯 지지율 변화나 득표율 예측 등을 집중 보도하...,⑤,"정답해설 : 1문단에서 경마식 보도가 선거와 정치에 무관심한 유권자들의 선거 참여,...",세부 내용 파악,국어,1_3
3,5. 윗글에서 알 수 있는 내용으로 적절하지 않은 것은?,①신뢰할 수 있는 여론조사의 결과를 보도하더라도 선거의 공정성을 위협할 수 있다.\...,㉠[경마식 보도]는 경마 중계를 하듯 지지율 변화나 득표율 예측 등을 집중 보도하...,③,정답해설 : 2문단에서 헌법재판소는 여론조사 결과의 보도를 일정 기간 금지하는 규\...,생략된 내용 추론,국어,1_4
4,6. ㉡과 관련하여 ⓐ와 ⓑ의 입장에 대한 반응으로 가장 적절한 것은? [3점],①선거 방송 초청 대상 후보자 토론회에서 후보자들이 심층적인 토론을 하지 못한 원인...,㉠[경마식 보도]는 경마 중계를 하듯 지지율 변화나 득표율 예측 등을 집중 보도하...,②,"정답해설 : 선거 방송 토론회의 초청 대상을 제한하는 「공직선거법」의 규정에 대해,...",다른 견해와의 비교,국어,1_5


In [8]:
import csv
import json
import re

# CSV 파일 경로
csv_file_path = '통합데이터.csv'
json_file_path = '통합데이터.json'

def clean_key(text):
    # 비표준 문자 제거 및 공백 문자 제거
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace characters with a single space
    return text.strip()

# CSV 파일 읽기
with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    
    # JSON 데이터 저장할 리스트
    json_data = []
    
    # CSV 각 행을 JSON 형식으로 변환하여 리스트에 추가
    for row in csv_reader:
        cleaned_row = {}
        for key, value in row.items():
            # 첫 번째 열의 키에서 공백 및 특수 문자 제거
            cleaned_key = clean_key(key)
            # answer 열의 값 변환
            if cleaned_key == 'answer':
                value = value.replace('①', '1').replace('②', '2').replace('③', '3').replace('④', '4').replace('⑤', '5')
            cleaned_row[cleaned_key] = value.strip()
        json_data.append(cleaned_row)

# JSON 파일로 저장
with open(json_file_path, 'w', encoding='utf-8') as jsonfile:
    json.dump(json_data, jsonfile, ensure_ascii=False, indent=4)

print("CSV 파일이 JSON 형식으로 변환되었습니다.")


CSV 파일이 JSON 형식으로 변환되었습니다.


In [1]:
import json
import mysql.connector

# JSON 파일 경로
json_file_path = '통합데이터.json'

# MySQL 연결 설정
db = mysql.connector.connect(
    host="43.203.121.11",     # MySQL 서버 호스트 이름
    user="aivle9",     # MySQL 사용자 이름
    password="aivle9", # MySQL 비밀번호
    database="aivle"   # 데이터베이스 이름
)

cursor = db.cursor()

# JSON 파일 열기
with open(json_file_path, 'r', encoding='utf-8') as jsonfile:
    data = json.load(jsonfile)
    
    # 데이터 삽입 쿼리
    query = """
    INSERT INTO subject (q_sub, question, selection, passage, answer, explanation, q_type, subject_name)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # JSON 데이터를 MySQL 테이블에 삽입
    for entry in data:
        cursor.execute(query, (
            entry["q_sub"],
            entry["question"],
            entry["selection"],
            entry["passage"],
            entry["answer"],
            entry["explanation"],
            entry["q_type"],
            entry["subject_name"],
        ))

# 변경사항 커밋
db.commit()

# 연결 종료
cursor.close()
db.close()

print("JSON 데이터를 MySQL 테이블에 성공적으로 삽입했습니다.")


JSON 데이터를 MySQL 테이블에 성공적으로 삽입했습니다.
